# MODELING

In [1]:
from tqdm import tqdm
from collections import defaultdict
import gensim
from gensim.corpora import Dictionary
from gensim.models import Phrases
from gensim.models import LdaModel
import pyLDAvis.gensim
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymystem3 import Mystem
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import seaborn as sns
%matplotlib inline

C:\Users\K\Anaconda3\lib\site-packages\gensim-2.2.0-py3.6-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\K\Anaconda3\lib\site-packages\geopandas\__init__.py:1: DeprecationWarning: pandas.util.decorators is deprecated and will be removed in a future version, import from pandas.util
  from geopandas.geoseries import GeoSeries


# Select the neighborhood and load its data

In [4]:
neigh_posts

NameError: name 'neigh_posts' is not defined

In [5]:
name = 'Izmajlovo'
district = gpd.read_file('../Data dive/dd2/{}/{}_district.geojson'.format(name,name))
neigh_posts = pd.read_csv('social_media/{}/vk.csv'.format(name))

In [3]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# Group posts by user

In [6]:
vk_users = pd.DataFrame({'post' : neigh_posts.groupby('userId').apply( lambda x: ' '.join(x['text']))}).reset_index()

# How many users are there?

In [7]:
extra_words = ['http','br','id','com','www', 'instagram', 'vsco', 'https', 'instasize','repost',
              'whatsapp', 'вотсап', 'repostapp','маникюр', 'бровь', 'ресница', 'губа', 'instacollage', 'опубликовывать',
                'фото', 'москва', 'moscow']
def process_docs(docs):
    """
    Function to process texts. Following are the steps we take:
    
    1. Text tokenization.
    2. Removing numbers 
    3. Stopword and short words Removal.
    4. Lemmatization and filter words by their length.
    
    Args:
    ----------
    texts: Tokenized texts.
    
    Returns:
    -------
    texts: Pre-processed tokenized texts.
    """
    m = Mystem()
    # Split the documents into tokens.
    tokenizer = RegexpTokenizer(r'\w+')
    stops = stopwords.words('russian') + stopwords.words('english') + extra_words
    
    for idx in tqdm(range(len(docs))):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not any(c.isdigit() for c in token) and ('id' not in token or 'club' not in token or 'ru' not in token)] for doc in tqdm(docs)]
    #Remove stopwords
    docs = [[token for token in doc if token not in stops] for doc in tqdm(docs)]
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in tqdm(docs)]
    return docs
    
def get_corpus(docs):
    
    """Add bigrams to docs and create corpus and dictionary for training
    
    Args:
        docs: list of tokenized and cleaned texts;
    Returns:
        corpus: list of lists of tuples, where first element of tuple is a word id
        and the second is the count of that word in the whole corpus
        dictionary: gensim.corpora.dictionary.Dictionary 
  
    """
    
    frequency = defaultdict(int)
    for text in tqdm(docs):
        for token in text:
            frequency[token] += 1

    texts = [[token for token in text if frequency[token] > 3] for text in tqdm(docs)]

    #Take the bigram, if token is a bigram, add to document.
    bigram = Phrases(texts, min_count = 20)
    for idx in tqdm(range(len(texts))):
        for token in bigram[texts[idx]]:
            if '_' in token:
                texts[idx].append(token)
    
    # Create a dictionary representation of the documents.
    dictionary = Dictionary(texts)
    # Filter out words that occur less than 20 documents, or more than 50% of the documents.
    #dictionary.filter_extremes(no_below=20, no_above=0.5)
    corpus = [dictionary.doc2bow(doc) for doc in tqdm(texts)]
    
    print('Number of unique tokens: {}'.format(len(dictionary)))
    print('Number of documents: {}'.format(len(corpus)))
    
    return corpus, dictionary

# Process texts to be ready for modeling

In [8]:
import nltk

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\K\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
texts = vk_users['post'].copy()

In [11]:
docs = process_docs(texts.values)

Installing mystem to C:\Users\K/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.0-win7-64bit.zip
100%|██████████| 17369/17369 [00:00<00:00, 71877.22it/s]


In [12]:
corpus, dictionary = get_corpus(docs)

  0%|          | 0/17369 [00:00<?, ?it/s]C:\Users\K\Anaconda3\lib\site-packages\gensim-2.2.0-py3.6-win-amd64.egg\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
100%|██████████| 17369/17369 [00:00<00:00, 56978.04it/s]


Number of unique tokens: 11207
Number of documents: 17369


In [13]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


#The training model - we use online LDA model which allows to update the model 
#and the following parameters should be defined
num_topics = 10 # number of topics
chunksize = 1000 
passes = 10
iterations = 400
eval_every = 10  #evaluate model perplexity.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha=0.001, update_every = 1, \
                       num_topics=num_topics,\
                       eval_every=eval_every, passes = passes)

2017-07-20 16:02:16,079 : INFO : using symmetric eta at 8.92299455697332e-05
2017-07-20 16:02:16,087 : INFO : using serial LDA version on this node
2017-07-20 16:02:16,812 : INFO : running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 17369 documents, updating model once every 1000 documents, evaluating perplexity every 10000 documents, iterating 50x with a convergence threshold of 0.001000
2017-07-20 16:02:16,815 : INFO : PROGRESS: pass 0, at document #1000/17369
2017-07-20 16:02:17,902 : INFO : merging changes from 1000 documents into a model of 17369 documents
2017-07-20 16:02:17,995 : INFO : topic #7 (0.001): 0.049*"измайлово" + 0.025*"парк" + 0.025*"измайловский" + 0.019*"кремль" + 0.017*"дельта" + 0.015*"измайлово_гамма" + 0.015*"гамма" + 0.014*"измайловский_парк" + 0.013*"кремль_измайлово" + 0.010*"rock"
2017-07-20 16:02:18,000 : INFO : topic #6 (0.001): 0.026*"измайлово" + 0.020*"hotel" + 0.016*"house" + 0.015*"vega" + 0.013*"ананас" + 0.013

2017-07-20 16:02:21,065 : INFO : merging changes from 1000 documents into a model of 17369 documents
2017-07-20 16:02:21,141 : INFO : topic #4 (0.001): 0.058*"измайловский" + 0.050*"измайлово" + 0.048*"усадьба" + 0.047*"остров" + 0.045*"измайловский_остров" + 0.041*"усадьба_измайлово" + 0.011*"проспект" + 0.011*"измайловский_проспект" + 0.008*"картинг" + 0.006*"музей_усадьба"
2017-07-20 16:02:21,144 : INFO : topic #8 (0.001): 0.029*"метро" + 0.017*"партизанская" + 0.013*"очень" + 0.009*"сегодня" + 0.008*"станция" + 0.007*"роддом" + 0.007*"метро_партизанская" + 0.007*"день" + 0.007*"алексино_истра" + 0.007*"станция_метро"
2017-07-20 16:02:21,147 : INFO : topic #1 (0.001): 0.036*"hotel" + 0.031*"vega" + 0.031*"best" + 0.029*"western" + 0.029*"best_western" + 0.029*"center" + 0.029*"convention" + 0.029*"plus" + 0.028*"hotel_convention" + 0.028*"plus_vega"
2017-07-20 16:02:21,150 : INFO : topic #5 (0.001): 0.094*"измайловский" + 0.075*"парк" + 0.068*"измайловский_парк" + 0.014*"бульвар" + 

2017-07-20 16:02:24,635 : INFO : topic #7 (0.001): 0.070*"измайлово" + 0.047*"дельта" + 0.046*"гамма" + 0.046*"измайлово_гамма" + 0.028*"park" + 0.024*"izmaylovsky" + 0.024*"izmaylovsky_park" + 0.020*"альфа" + 0.020*"гостиница" + 0.020*"сокольники"
2017-07-20 16:02:24,638 : INFO : topic #3 (0.001): 0.053*"ргуфксит" + 0.018*"ргуфксмит" + 0.017*"izmaylovo" + 0.015*"district" + 0.015*"izmaylovo_district" + 0.012*"pervomayskaya" + 0.011*"ргуфк" + 0.010*"beta" + 0.009*"hotel" + 0.009*"измайлово"
2017-07-20 16:02:24,641 : INFO : topic #0 (0.001): 0.160*"измайлово" + 0.145*"кремль" + 0.107*"кремль_измайлово" + 0.025*"измайловский" + 0.018*"площадь" + 0.018*"красная" + 0.017*"красная_площадь" + 0.016*"измайловскийкремль" + 0.009*"виноградный" + 0.008*"серебряно"
2017-07-20 16:02:24,643 : INFO : topic #6 (0.001): 0.034*"hotel" + 0.024*"make" + 0.024*"atelier" + 0.021*"make_atelier" + 0.020*"school" + 0.019*"delta" + 0.014*"gamma" + 0.013*"gamma_delta" + 0.013*"спасибо" + 0.011*"дельта_izmailovo

2017-07-20 16:02:26,949 : INFO : topic #5 (0.001): 0.106*"измайловский" + 0.084*"парк" + 0.071*"измайловский_парк" + 0.018*"бульвар" + 0.017*"измайловский_бульвар" + 0.014*"первомайская" + 0.012*"измайловскийпарк" + 0.009*"гимназия" + 0.007*"отдыха" + 0.006*"культуры"
2017-07-20 16:02:26,954 : INFO : topic diff=0.301345, rho=0.250000
2017-07-20 16:02:26,956 : INFO : PROGRESS: pass 0, at document #17000/17369
2017-07-20 16:02:27,330 : INFO : merging changes from 1000 documents into a model of 17369 documents
2017-07-20 16:02:27,378 : INFO : topic #5 (0.001): 0.100*"измайловский" + 0.079*"парк" + 0.067*"измайловский_парк" + 0.017*"бульвар" + 0.017*"первомайская" + 0.016*"измайловский_бульвар" + 0.016*"пишите" + 0.013*"вопросам" + 0.011*"всем" + 0.010*"измайловскийпарк"
2017-07-20 16:02:27,380 : INFO : topic #3 (0.001): 0.053*"ргуфксит" + 0.021*"izmaylovo" + 0.020*"izmaylovo_district" + 0.020*"district" + 0.018*"pervomayskaya" + 0.017*"beta" + 0.014*"ргуфксмит" + 0.011*"ргуфк" + 0.011*"ho

2017-07-20 16:02:29,535 : INFO : topic #3 (0.001): 0.038*"ргуфксит" + 0.022*"izmaylovo" + 0.022*"district" + 0.021*"izmaylovo_district" + 0.016*"pervomayskaya" + 0.011*"ргуфксмит" + 0.009*"beta" + 0.009*"ргуфк" + 0.008*"всем" + 0.007*"парикмахерская"
2017-07-20 16:02:29,538 : INFO : topic #4 (0.001): 0.066*"измайловский" + 0.054*"измайлово" + 0.051*"остров" + 0.051*"усадьба" + 0.049*"измайловский_остров" + 0.044*"усадьба_измайлово" + 0.019*"проспект" + 0.019*"измайловский_проспект" + 0.008*"платье" + 0.008*"картинг"
2017-07-20 16:02:29,542 : INFO : topic #0 (0.001): 0.167*"измайлово" + 0.145*"кремль" + 0.110*"кремль_измайлово" + 0.024*"измайловский" + 0.015*"измайловскийкремль" + 0.013*"площадь" + 0.012*"красная" + 0.011*"красная_площадь" + 0.011*"свадьба" + 0.008*"вернисаж"
2017-07-20 16:02:29,544 : INFO : topic #9 (0.001): 0.015*"магазин" + 0.014*"цена" + 0.012*"наличии" + 0.009*"очень" + 0.008*"концертный" + 0.008*"концертный_измайлово" + 0.007*"измайлово" + 0.007*"заказа" + 0.007*"

...

2017-07-20 16:03:27,475 : INFO : PROGRESS: pass 8, at document #17000/17369
2017-07-20 16:03:27,769 : INFO : merging changes from 1000 documents into a model of 17369 documents
2017-07-20 16:03:27,796 : INFO : topic #0 (0.001): 0.187*"измайлово" + 0.157*"кремль" + 0.121*"кремль_измайлово" + 0.023*"площадь" + 0.023*"измайловский" + 0.021*"красная" + 0.019*"красная_площадь" + 0.013*"измайловскийкремль" + 0.010*"вернисаж" + 0.007*"опубликовано"
2017-07-20 16:03:27,800 : INFO : topic #8 (0.001): 0.039*"метро" + 0.023*"партизанская" + 0.011*"метро_партизанская" + 0.011*"станция" + 0.010*"спасибо" + 0.010*"partizanskaya" + 0.010*"очень" + 0.009*"сегодня" + 0.007*"день" + 0.007*"станция_метро"
2017-07-20 16:03:27,807 : INFO : topic #9 (0.001): 0.025*"всем" + 0.022*"цена" + 0.021*"заказа" + 0.015*"первомайская" + 0.014*"square" + 0.013*"цена_всем" + 0.013*"вопросам_пишите" + 0.009*"концертный" + 0.008*"мюзикл" + 0.008*"очень"
2017-07-20 16:03:27,810 : INFO : topic #2 (0.001): 0.017*"izmailovo"

2017-07-20 16:03:29,731 : INFO : topic #4 (0.001): 0.066*"измайловский" + 0.057*"измайлово" + 0.051*"усадьба" + 0.050*"остров" + 0.048*"измайловский_остров" + 0.043*"усадьба_измайлово" + 0.017*"проспект" + 0.017*"измайловский_проспект" + 0.013*"платьев" + 0.013*"магазин_платьев"
2017-07-20 16:03:29,737 : INFO : topic #8 (0.001): 0.028*"метро" + 0.021*"партизанская" + 0.013*"очень" + 0.012*"сегодня" + 0.011*"partizanskaya" + 0.009*"спасибо" + 0.008*"метро_партизанская" + 0.007*"день" + 0.007*"станция" + 0.006*"рождения"
2017-07-20 16:03:29,741 : INFO : topic #9 (0.001): 0.016*"всем" + 0.014*"магазин" + 0.014*"цена" + 0.011*"наличии" + 0.009*"очень" + 0.009*"первомайская" + 0.008*"концертный" + 0.008*"square" + 0.008*"концертный_измайлово" + 0.007*"заказа"
2017-07-20 16:03:29,743 : INFO : topic #5 (0.001): 0.118*"измайловский" + 0.086*"парк" + 0.073*"измайловский_парк" + 0.019*"бульвар" + 0.014*"измайловский_бульвар" + 0.014*"измайловскийпарк" + 0.008*"отдыха" + 0.008*"культуры" + 0.007*

2017-07-20 16:03:31,529 : INFO : topic #9 (0.001): 0.013*"всем" + 0.011*"первомайская" + 0.011*"концертный" + 0.010*"square" + 0.010*"концертный_измайлово" + 0.008*"очень" + 0.007*"мюзикл" + 0.007*"спасибо" + 0.007*"магазин" + 0.007*"качка"
2017-07-20 16:03:31,532 : INFO : topic #1 (0.001): 0.068*"hotel" + 0.055*"vega" + 0.047*"best" + 0.044*"western" + 0.044*"best_western" + 0.044*"center" + 0.044*"convention" + 0.042*"plus" + 0.042*"plus_vega" + 0.042*"hotel_convention"
2017-07-20 16:03:31,538 : INFO : topic diff=0.049655, rho=0.191148
2017-07-20 16:03:32,300 : INFO : -8.298 per-word bound, 314.8 perplexity estimate based on a held-out corpus of 1000 documents with 14772 words
2017-07-20 16:03:32,303 : INFO : PROGRESS: pass 9, at document #10000/17369
2017-07-20 16:03:32,622 : INFO : merging changes from 1000 documents into a model of 17369 documents
2017-07-20 16:03:32,645 : INFO : topic #8 (0.001): 0.026*"метро" + 0.017*"партизанская" + 0.013*"сегодня" + 0.012*"очень" + 0.012*"спас

2017-07-20 16:03:34,093 : INFO : topic diff=0.051234, rho=0.191148
2017-07-20 16:03:34,096 : INFO : PROGRESS: pass 9, at document #15000/17369
2017-07-20 16:03:34,407 : INFO : merging changes from 1000 documents into a model of 17369 documents
2017-07-20 16:03:34,435 : INFO : topic #4 (0.001): 0.070*"измайлово" + 0.060*"измайловский" + 0.058*"усадьба" + 0.056*"остров" + 0.054*"измайловский_остров" + 0.048*"усадьба_измайлово" + 0.011*"музей" + 0.009*"музей_усадьба" + 0.009*"картинг" + 0.009*"vscocam"
2017-07-20 16:03:34,441 : INFO : topic #8 (0.001): 0.022*"метро" + 0.013*"партизанская" + 0.012*"очень" + 0.011*"спасибо" + 0.010*"сегодня" + 0.010*"partizanskaya" + 0.008*"холи" + 0.007*"измайловского" + 0.007*"день" + 0.007*"метро_партизанская"
2017-07-20 16:03:34,444 : INFO : topic #1 (0.001): 0.074*"hotel" + 0.057*"vega" + 0.049*"best" + 0.047*"western" + 0.047*"best_western" + 0.046*"center" + 0.046*"convention" + 0.045*"plus" + 0.044*"plus_vega" + 0.044*"hotel_convention"
2017-07-20 1

Wall time: 1min 19s


In [ ]:
# import nltk
# nltk.download()

In [14]:
data = pyLDAvis.gensim.prepare(model, corpus, dictionary); # visualize lda topics
pyLDAvis.display(data)

C:\Users\K\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


# Assign topics to users

In [15]:
docTopicProbMat = model[corpus]
lda_users = vk_users.copy()
lda_users['topics'] = docTopicProbMat
vk_users['topic'] = lda_users['topics'].apply(lambda x :x[0][0])

# Assign topics to their posts

In [16]:
neigh_posts['topic'] = neigh_posts['userId'].apply(lambda userId: vk_users.loc[vk_users['userId']==userId,'topic'].item())

# Plot the histogram of topic distribution per posts and per users

# Mapping

In [17]:
center_lat = list(district.centroid[0].coords)[0][1]
center_lon = list(district.centroid[0].coords)[0][0]

In [18]:
map_places = folium.Map([center_lat, center_lon], tiles='Stamen Toner', zoom_start=14,control_scale=True)

#Define style for geojson objects
style_function = lambda feature: dict(fillColor='#AECCAE',
                                      color='#AECCAE',
                                      weight=1,
                                      opacity=0.3)

# Adding Houses
houses = gpd.read_file('../Data dive/dd2/{}/{}_chruchevki.geogson'.format(name, name))
points = folium.features.GeoJson(houses,name='Khurshevki houses')
map_places.add_child(points,name='Khurshevki houses')

#Adding district
polygon = folium.features.GeoJson(district, style_function=style_function,name='district boundary')
map_places.add_child(polygon,name='district boundary')

colormap_dict = {.0: 'blue', .2: 'cyan', .4: 'green', .6: 'yellow', .8:'orange', 1.:'red'}

#Adding topics heatmaps
for topic_id in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    topic_coords = list(zip(neigh_posts[neigh_posts.topic == topic_id].lat, neigh_posts[neigh_posts.topic == topic_id].lon))

    HeatMap(topic_coords,
            name='Topic: {}'.format(topic_id),
            radius=10, 
            min_opacity=0.8,
            gradient={.0: 'blue', .2: 'cyan', .4: 'green', .6: 'yellow', .8:'orange', 1.:'red'}).add_to(map_places)


    colormap = folium.LinearColormap(colors = colormap_dict.values())
    colormap.caption = 'Topic: {}'.format(topic_id)

#Switch between layers
folium.LayerControl().add_to(map_places)
map_places